In [ ]:
import datetime

import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import datetime

df = pd.read_csv("../accelerometer-data-2023-08-25 20:31:42.558559.csv")
df["timestamp"] = pd.to_datetime(
    df["timestamp"].map(lambda x: datetime.datetime.fromtimestamp(x))
)
df = df.set_index("timestamp")
df.head()

In [ ]:
df.plot()
plt.legend(loc="upper left")
plt.title("Accelerometer Data")
plt.grid(True)
plt.ylim(-4, 4)
plt.ylabel("Acceleration (G)")
plt.show()
# # plot x y z on a top plot and button_state on a bottom plot, share x axis
# fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 6))
# df.plot(y=['x', 'y', 'z'], ax=ax1, label=['x', 'y', 'z'])
# ax1.legend(loc="upper left")
# ax1.set_title("Accelerometer Data")
# ax1.grid(True)
# ax1.set_ylim(-4, 4)
# ax1.set_ylabel("Acceleration (G)")
# df.plot(y=['button_state'], ax=ax2, label=['button_state'])
# ax2.legend(loc="upper left")
# ax2.set_ylim(-0.1, 1.1)
# ax2.set_ylabel("Button State")
# plt.show()

In [ ]:
# plot magnitude of acceleration vector and button state
df["magnitude"] = (df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2) ** 0.5
df.plot(y=["magnitude", "button_state"], label=["magnitude", "button_state"])
plt.show()

In [ ]:
X = df[["x", "y", "z", "magnitude"]].values
y = df["button_state"].values

In [ ]:
# preprocessing
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, class_weight="balanced")
scores = cross_val_score(clf, X, y, cv=5, scoring="f1_macro")
scores

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

y_pred = cross_val_predict(clf, X, y, cv=5, method="predict_proba")
conf_mat = confusion_matrix(y, y_pred[:, 1] > 0.5, normalize="true")
print(conf_mat)

disp = ConfusionMatrixDisplay(
    confusion_matrix=conf_mat, display_labels=["nothing", "step"]
)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report

target_names = ["nothing", "step"]
print(classification_report(y, y_pred[:, 1] > 0.5, target_names=target_names))

In [ ]:
# plot roc curve
import matplotlib.pyplot as plt

from sklearn.metrics import DetCurveDisplay, RocCurveDisplay

fig, ax_roc = plt.subplots(1, 1, figsize=(5, 5))
RocCurveDisplay.from_predictions(y, y_pred[:, 1], ax=ax_roc)
ax_roc.set_title("Receiver Operating Characteristic (ROC) curves")
ax_roc.grid(linestyle="--")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve

fig, ax_pr = plt.subplots(1, 1, figsize=(5, 5))
prec, recall, _ = precision_recall_curve(y, y_pred[:, 1], pos_label=1)
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot(ax=ax_pr)
ax_pr.grid(linestyle="--")
plt.show()

In [ ]:
plt.plot(y - (y_pred[:, 1] > 0.5))